## Exercícios

> Retirados de [learn-python: sqlalchemy_orm-questions](https://aviadr1.github.io/learn-advanced-python/11_db_access/exercise/sqlalchemy_orm-questions.html).

#### Q1.

Baixa e extraia o arquivo compactado com o banco de dados [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/). Salve o arquivo `chinook.db` na mesma pasta deste script.
* Link para baixar: http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

<img width=500 src=https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg>


#### Q2.

Leia o código e os comentários das células a seguir para entender como acessamos os modelos ORM de um banco já existente.

In [8]:
!pip install sqlalchemy

  Using cached sqlalchemy-2.0.44-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.5 kB)
  Using cached greenlet-3.2.4-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (4.1 kB)
Using cached sqlalchemy-2.0.44-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
Using cached greenlet-3.2.4-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl (607 kB)


In [9]:
import sqlalchemy
print(sqlalchemy.__version__)

2.0.44


In [10]:
from sqlalchemy import create_engine, text, MetaData
from sqlalchemy.orm import Session

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

### extrai as classes da base de dados Chinook
metadata = MetaData()
metadata.reflect(engine)

# O metadata tem informações sobre as tabelas
# que serão usadas para criar os modelos ORM
for table_name, table in metadata.tables.items():
    print(table_name)
    print(table.columns.keys())
    print(table.columns.items())
    print('-'*25)

### configura o objeto Base mapeando os modelos ORM das tabelas
from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

# o objeto Base tem os modelos ORM que podemos usar
# para manipular o banco de dados
print(Base.classes.items())

albums
['AlbumId', 'Title', 'ArtistId']
[('AlbumId', Column('AlbumId', INTEGER(), table=<albums>, primary_key=True, nullable=False)), ('Title', Column('Title', NVARCHAR(length=160), table=<albums>, nullable=False)), ('ArtistId', Column('ArtistId', INTEGER(), ForeignKey('artists.ArtistId'), table=<albums>, nullable=False))]
-------------------------
artists
['ArtistId', 'Name']
[('ArtistId', Column('ArtistId', INTEGER(), table=<artists>, primary_key=True, nullable=False)), ('Name', Column('Name', NVARCHAR(length=120), table=<artists>))]
-------------------------
customers
['CustomerId', 'FirstName', 'LastName', 'Company', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email', 'SupportRepId']
[('CustomerId', Column('CustomerId', INTEGER(), table=<customers>, primary_key=True, nullable=False)), ('FirstName', Column('FirstName', NVARCHAR(length=40), table=<customers>, nullable=False)), ('LastName', Column('LastName', NVARCHAR(length=20), table=<customers>, nullable=F

In [14]:
# A seguir um exemplo de query na tabela Albums
# usamos o objeto Base para acessar cada modelo ORM.

session = Session(engine)
res = session.scalars(select(Base.classes.albums))
first_album = res.first()
print(first_album.AlbumId, first_album.Title)

1 For Those About To Rock We Salute You


In [13]:
from sqlalchemy import create_engine, select, MetaData, func
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base

# Caminho do banco (precisa estar na mesma pasta do notebook)
engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

# Reflete as tabelas do banco (lê a estrutura existente)
metadata = MetaData()
metadata.reflect(engine)

# Cria classes ORM automaticamente
Base = automap_base(metadata=metadata)
Base.prepare()

# Cria sessão
session = Session(engine)

# Mapeia tabelas
Tracks = Base.classes.tracks
Albums = Base.classes.albums
InvoiceItems = Base.classes.invoice_items
Artists = Base.classes.artists

print("✅ Banco e tabelas carregados com sucesso!")


✅ Banco e tabelas carregados com sucesso!


#### Q3. 
Com base nos códigos anteriores realize as operações solicitadas nas células a seguir:


In [15]:
### Imprima os três primeiros registros da tabela tracks

from sqlalchemy import create_engine, select, MetaData
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base

# Conecta ao banco Chinook
engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

# Mapeia as tabelas automaticamente
metadata = MetaData()
metadata.reflect(engine)
Base = automap_base(metadata=metadata)
Base.prepare()

# Inicia sessão
session = Session(engine)

# Classe ORM para a tabela tracks
Tracks = Base.classes.tracks

# Consulta: primeiros 3 registros
res = session.execute(select(Tracks).limit(3))
for row in res.scalars():
    print(f"TrackId={row.TrackId}, Name={row.Name}, AlbumId={row.AlbumId}, Composer={row.Composer}")

TrackId=1, Name=For Those About To Rock (We Salute You), AlbumId=1, Composer=Angus Young, Malcolm Young, Brian Johnson
TrackId=2, Name=Balls to the Wall, AlbumId=2, Composer=None
TrackId=3, Name=Fast As a Shark, AlbumId=3, Composer=F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman


In [16]:
### Imprima o nome da faixa e o título do álbum das primeiras 20 faixas na tabela tracks.

Albums = Base.classes.albums

# Join entre tracks e albums
stmt = (
    select(Tracks.Name, Albums.Title)
    .join(Albums, Tracks.AlbumId == Albums.AlbumId)
    .limit(20)
)

for name, title in session.execute(stmt):
    print(f"Faixa: {name} | Álbum: {title}")

Faixa: For Those About To Rock (We Salute You) | Álbum: For Those About To Rock We Salute You
Faixa: Put The Finger On You | Álbum: For Those About To Rock We Salute You
Faixa: Let's Get It Up | Álbum: For Those About To Rock We Salute You
Faixa: Inject The Venom | Álbum: For Those About To Rock We Salute You
Faixa: Snowballed | Álbum: For Those About To Rock We Salute You
Faixa: Evil Walks | Álbum: For Those About To Rock We Salute You
Faixa: C.O.D. | Álbum: For Those About To Rock We Salute You
Faixa: Breaking The Rules | Álbum: For Those About To Rock We Salute You
Faixa: Night Of The Long Knives | Álbum: For Those About To Rock We Salute You
Faixa: Spellbound | Álbum: For Those About To Rock We Salute You
Faixa: Balls to the Wall | Álbum: Balls to the Wall
Faixa: Fast As a Shark | Álbum: Restless and Wild
Faixa: Restless and Wild | Álbum: Restless and Wild
Faixa: Princess of the Dawn | Álbum: Restless and Wild
Faixa: Go Down | Álbum: Let There Be Rock
Faixa: Dog Eat Dog | Álbum: Le

In [17]:
### Imprima as 10 primeiras vendas de faixas da tabela invoice_items
### Para essas 10 primeiras vendas, imprima os nomes das faixas vendidas e a quantidade vendida.

InvoiceItems = Base.classes.invoice_items

# Join entre invoice_items e tracks
stmt = (
    select(Tracks.Name, InvoiceItems.Quantity)
    .join(Tracks, InvoiceItems.TrackId == Tracks.TrackId)
    .limit(10)
)

for name, qty in session.execute(stmt):
    print(f"Faixa vendida: {name} | Quantidade: {qty}")

Faixa vendida: Balls to the Wall | Quantidade: 1
Faixa vendida: Restless and Wild | Quantidade: 1
Faixa vendida: Put The Finger On You | Quantidade: 1
Faixa vendida: Inject The Venom | Quantidade: 1
Faixa vendida: Evil Walks | Quantidade: 1
Faixa vendida: Breaking The Rules | Quantidade: 1
Faixa vendida: Dog Eat Dog | Quantidade: 1
Faixa vendida: Overdose | Quantidade: 1
Faixa vendida: Love In An Elevator | Quantidade: 1
Faixa vendida: Janie's Got A Gun | Quantidade: 1


In [18]:
### Imprima os nomes das 10 faixas mais vendidas e quantas vezes foram vendidas.

from sqlalchemy import func

# Agrupa e conta vendas por faixa
stmt = (
    select(Tracks.Name, func.sum(InvoiceItems.Quantity).label("total_vendas"))
    .join(InvoiceItems, Tracks.TrackId == InvoiceItems.TrackId)
    .group_by(Tracks.Name)
    .order_by(func.sum(InvoiceItems.Quantity).desc())
    .limit(10)
)

print("Top 10 faixas mais vendidas:\n")
for name, total in session.execute(stmt):
    print(f"{name}: {total} vendas")

Top 10 faixas mais vendidas:

The Trooper: 5 vendas
Untitled: 4 vendas
The Number Of The Beast: 4 vendas
Sure Know Something: 4 vendas
Hallowed Be Thy Name: 4 vendas
Eruption: 4 vendas
Where Eagles Dare: 3 vendas
Welcome Home (Sanitarium): 3 vendas
Sweetest Thing: 3 vendas
Surrender: 3 vendas


In [19]:
### Quem são os 10 artistas que mais venderam?
### dica: você precisa juntar as tabelas invoice_items, tracks, albums e artists

Artists = Base.classes.artists

stmt = (
    select(Artists.Name, func.sum(InvoiceItems.Quantity).label("total_vendas"))
    .join(Albums, Artists.ArtistId == Albums.ArtistId)
    .join(Tracks, Albums.AlbumId == Tracks.AlbumId)
    .join(InvoiceItems, Tracks.TrackId == InvoiceItems.TrackId)
    .group_by(Artists.Name)
    .order_by(func.sum(InvoiceItems.Quantity).desc())
    .limit(10)
)

print("Top 10 artistas com mais faixas vendidas:\n")
for artist, total in session.execute(stmt):
    print(f"{artist}: {total} vendas")

Top 10 artistas com mais faixas vendidas:

Iron Maiden: 140 vendas
U2: 107 vendas
Metallica: 91 vendas
Led Zeppelin: 87 vendas
Os Paralamas Do Sucesso: 45 vendas
Deep Purple: 44 vendas
Faith No More: 42 vendas
Lost: 41 vendas
Eric Clapton: 40 vendas
R.E.M.: 39 vendas
